# Phone Loop Creation

This notebook illustrate how to use [AMDTK](https://github.com/iondel/amdtk) to create a Bayesian Phone Loop model.

## Setting up the environment

Basic preliminary operations (importing modules, configuration, ...)

In [1]:
import os
import pickle
import numpy as np
import amdtk
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

Some operation are parallelized with [ipyparallel](https://ipyparallel.readthedocs.io/en/latest), make sure the ipyparallel is installed and properly configured. If `profile = None` the default (local) parallel environment will be used.

In [12]:
from ipyparallel import Client
profile = None
rc = Client(profile=profile)
dview = rc[:]
print('# jobs:', len(dview))

# jobs: 4


## Prepare the database

Prepare the database from which we would like to infer the phone-like units. The input is a list of features files named according to the following convention:
```
/path/to/listdir/dbname_featype.scp
```
For instance if you database is the training set of TIMIT and you use MFCC features, we expect the list of features to be named:
```
/path/to/listdir/timit_train_mfcc.scp
```
The list of features files should be formatted as follow: 
```
UNIQUE_KEY1 /path/to/features/file1.fea
UNIQUE_KEY2 /path/to/features/file2.fea
...
```

If only parts of some of the features files are to be processed (for example if the audio contains speech and silence) the portion of features to be loaded can be specified by:
```
UNIQUE_KEY1 /path/to/features/file1.fea[start:end]
UNIQUE_KEY2 /path/to/features/file2.fea
UNIQUE_KEY3 /path/to/features/file3.fea
...
```
where `start` and `end` are the index of the sequence of frames of interest. Note that the frame indexed by "`end`" will not be included. 

In this notebook, the data is used only for initialization. if your database is large, it is probably better to use only a subset for estimating the desired statistics. You can reduce the number of speech segments to 1000 by adding
```
segments = segments[:1000]
```
at the end of the cell.

In [9]:
listdir = '/mnt/matylda5/iondel/workspace/features/lists'
dbname = 'timit_train'
featype = 'mfcc'
path = os.path.join(listdir, dbname + '_' + featype + '.scp')
segments = []
segments_key = {}
with open(path, 'r') as f:
    for line in f:
        key, segment = line.strip().split()
        segments.append(segment)
        segments_key[segment] = key

## Database statistics

For the initialization of the model we will need the mean and the diagonal of the covariance matrix of the data set.

In [10]:
def data_stats(filename):
    """Thread to collect the sufficients statistics for the mean and the
    covariance matrix from a set of features.
    
    """
    import amdtk
    data = amdtk.read_htk(filename)
    stats_0 = data.shape[0]
    stats_1 = data.sum(axis=0)
    stats_2 = (data**2).sum(axis=0)
    stats = {}
    stats[0] = {
        'N': stats_0,
        'x': stats_1,
        'x2': stats_2
    }
    return stats
    
stats = dview.map_sync(data_stats, segments)
stats = amdtk.acc_stats(stats)
N = stats[0]['N']
db_mean = stats[0]['x'] / N
db_prec = N / stats[0]['x2']

# Sanity check, plot the mean/std_dev of the data. 
x_data = np.arange(len(db_mean))
std_deviation = np.sqrt(1/db_prec)
p = figure(title='Mean (+- std) of the features per dimension',
           x_axis_label='dimension',
           y_axis_label='Mean',
           height=400, width=400)
p.x_range.range_padding = 0
p.line(x_data, db_mean, line_width=1.5)
upper_band = db_mean + std_deviation
lower_band = db_mean - std_deviation
p.patch(np.append(x_data, x_data[::-1]), 
        np.append(lower_band, upper_band[::-1]),
        fill_alpha=0.2)
show(p)

## Create the model
Create the phone loop model. By default, all the parameters of the posterior distributions, except for the posterior of Gaussian's means, are initialized to their corresponding prior. The Gaussian's posterior means are sampled from a Gaussian distribution fitted to the whole database. Here is the list of parameters:
  * `n_units`: maximum number of acoustic units
  * `n_states`: number of states insides a units
  * `n_gauss`: number of Gaussian per state
  * `prior_mean`: *a priori*  mean of the Gaussian components. When lacking of information a "good" choice is to set it to the global mean of the data
  * `prior_mcount`: pseudo-count of the prior mean, low/high value corresponds to weak/strong prior. Should be greater than 0
  * `prior_prec`: *a priori* precision (inverse variance) of the Gaussian components. When lacking of information a good choice is to set it to the global precision of the database. It has to be the same dimension as `prior_mean`
  * `prior_pcount`: pseudo-count of the prior precision, low/high value corresponds to weak/strong prior. Should be greater than 0
  * `prior_pcount`: pseudo-count of each Gaussian component for each state, low/high value corresponds to weak/strong prior. Should be greater than 0
  * `concentration`: concentration parameter for the Dirichlet Process/Distribution
  * `dp_prior`: if True, use a Dirichlet Process prior over the weights for the units. Use a Dirichlet distribution otherwise.
  * `ins_penalty`: insertion penalty 

In [6]:
n_units = 100
n_states = 3
n_gauss = 2
prior_mean = db_mean
prior_mcount = 1.
prior_prec = db_prec
prior_pcount = 1.
prior_wcount = np.zeros(n_gauss) + 1.
concentration = 1e-3
dp_prior = False
ins_penalty = 1.

# Create the state emissions.
emissions = []
for i in range(n_units * n_states):
    gaussians = [amdtk.GaussianDiagCov(prior_mean, 
                                       prior_mcount, 
                                       prior_prec, 
                                       prior_pcount) 
                 for i in range(n_gauss)]
    emissions.append(amdtk.Mixture(gaussians, prior_wcount))

# Create the phone loop model.
init_ploop = amdtk.PhoneLoop(n_units, 
                             emissions, 
                             concentration, 
                             ins_penalty, 
                             dp_prior)

# Initialize the Gaussian mean's posterior.
for units in init_ploop.components:
    for g in units.components:
        g.pmean = np.random.multivariate_normal(db_mean, np.diag(1/db_prec))

## Save the model
This series of examples will store all the models as:
```
models/ploop_[params]/[type]
```
where `[params]` are the specific parameters of the models (database, maximum numbers of units, number of states per units, ...) separated by an underscore and `[type]` indicates how the model was trained/initialized. In this example, the type will be `randInit` because the model was randomly initialized.

Note: to prevent overriding existing model, this code will fail if the directory already exists.

In [8]:
outdir_name = 'ploop_' + dbname + '_'          # database name
outdir_name += 'pDP_' if dp_prior else 'pD_'   # weights prior
outdir_name += 'c' + str(concentration) + '_'  # concentration
outdir_name += 'u' + str(n_units) + '_'        # number of units
outdir_name += 's' + str(n_states) + '_'       # number of state per units
outdir_name += 'g' + str(n_gauss)              # number of Gaussian per state

cwd = os.getcwd()
model_dir = os.path.join(cwd, 'models', outdir_name, 'randInit')
os.makedirs(model_dir)
with open(os.path.join(model_dir, 'model.bin'), 'wb') as f:
    pickle.dump(init_ploop, f)
print('output directory:', model_dir)

FileExistsError: [Errno 17] File exists: '/mnt/matylda5/iondel/workspace/amdtk/models/ploop_timit_train_pD_c0.001_u100_s3_g2/randInit'